# Import 

In [11]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Read

In [2]:

train = pl.read_csv('../data/input/train_ver2.csv', null_values=[' NA', 'NA','     NA', 'P'])
test = pl.read_csv('../data/input/test_ver2.csv', null_values=[' NA', 'NA','     NA'])

/home/ubuntu/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (5,8,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# # カラムのデータ型に不安があるときはpandasのほうが便利
# # DtypeWarningでカラム列を指定してくれた上でとりあえず読み込んでくれる

# trn = pd.read_csv('../data/input/train_ver2.csv')

# print(trn.iloc[:,[5,8,11,15]].columns)# mixed types columns

In [4]:
# # 試しにconyuempの中身を確認

# print(trn.conyuemp.unique())

# trn.groupby('conyuemp')['ncodpers'].count()

# 学習データ

In [5]:
print(train.shape)
train.head(3)

(13647309, 48)


fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
str,i64,str,str,str,i64,str,i64,i64,i64,str,f64,str,str,str,str,str,str,i64,i64,str,i64,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""2015-01-28""",1375586,"""N""","""ES""","""H""",35,"""2015-01-12""",0,6,1,null,1.0,"""A""","""S""","""N""",null,"""KHL""","""N""",1,29,"""MALAGA""",1,87218.1,"""02 - PARTICULA...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2015-01-28""",1050611,"""N""","""ES""","""V""",23,"""2012-08-10""",0,35,1,null,1.0,"""I""","""S""","""S""",null,"""KHE""","""N""",1,13,"""CIUDAD REAL""",0,35548.74,"""03 - UNIVERSIT...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""2015-01-28""",1050612,"""N""","""ES""","""V""",23,"""2012-08-10""",0,35,1,null,1.0,"""I""","""S""","""N""",null,"""KHE""","""N""",1,13,"""CIUDAD REAL""",0,122179.11,"""03 - UNIVERSIT...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
print(train.glimpse())

Rows: 13647309
Columns: 48
$ fecha_dato            <str> 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28, 2015-01-28
$ ncodpers              <i64> 1375586, 1050611, 1050612, 1050613, 1050614, 1050615, 1050616, 1050617, 1050619, 1050620
$ ind_empleado          <str> N, N, N, N, N, N, N, N, N, N
$ pais_residencia       <str> ES, ES, ES, ES, ES, ES, ES, ES, ES, ES
$ sexo                  <str> H, V, V, H, V, H, H, H, H, H
$ age                   <i64> 35, 23, 23, 22, 23, 23, 23, 23, 24, 23
$ fecha_alta            <str> 2015-01-12, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10, 2012-08-10
$ ind_nuevo             <i64> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
$ antiguedad            <i64> 6, 35, 35, 35, 35, 35, 35, 35, 35, 35
$ indrel                <i64> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
$ ult_fec_cli_1t        <str> None, None, None, None, None, None, None, None, None, None
$ indrel_1mes           <

In [7]:
num_col = [col for col in train.columns[:24] if str(train[col].dtype) in ['Int64','Float64']]
print(num_col)

cats_col = [col for col in train.columns[:24] if str(train[col].dtype) in ['Utf8']] # str型はUtf8で表現される
print(cats_col)

['ncodpers', 'age', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tipodom', 'cod_prov', 'ind_actividad_cliente', 'renta']
['fecha_dato', 'ind_empleado', 'pais_residencia', 'sexo', 'fecha_alta', 'ult_fec_cli_1t', 'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'segmento']


In [8]:
train.select(num_col).describe()

describe,ncodpers,age,ind_nuevo,antiguedad,indrel,indrel_1mes,tipodom,cod_prov,ind_actividad_cliente,renta
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7,1.3647309e7
"""null_count""",0.0,27734.0,27734.0,27734.0,27734.0,150655.0,27735.0,93591.0,27734.0,2.794375e6
"""mean""",834904.211501,40.181465,0.059562,76.591941,1.178399,1.00081,1.0,26.57147,0.457811,134254.318238
"""std""",431565.025784,17.185087,0.236673,1671.806547,4.177469,0.039877,0.0,12.784017,0.498217,230620.238822
"""min""",15889.0,2.0,0.0,-9.99999e5,1.0,1.0,1.0,1.0,0.0,1202.73
"""max""",1.553689e6,164.0,1.0,256.0,99.0,4.0,1.0,52.0,1.0,2.8894e7
"""median""",931893.0,39.0,0.0,50.0,1.0,1.0,1.0,28.0,0.0,101850.0


In [9]:
train.select(cats_col).describe()

describe,fecha_dato,ind_empleado,pais_residencia,sexo,fecha_alta,ult_fec_cli_1t,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,nomprov,segmento
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""count""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309""","""13647309"""
"""null_count""","""0""","""27734""","""27734""","""27804""","""27734""","""13622516""","""154437""","""27734""","""27734""","""13645501""","""186126""","""27734""","""0""","""189368"""
"""mean""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""min""","""2015-01-28""","""A""","""AD""","""H""","""1995-01-16""","""2015-07-01""","""A""","""N""","""N""","""N""","""004""","""N""","""""","""01 - TOP"""
"""max""","""2016-05-28""","""S""","""ZW""","""V""","""2016-05-31""","""2016-05-30""","""R""","""S""","""S""","""S""","""RED""","""S""","""ZARAGOZA""","""03 - UNIVERSIT..."
"""median""",null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [17]:
# カテゴリー値の固有値を直接出力

for col in cats_col:
    uniq = np.unique(train.select(col))
    print('-' * 50)
    print('# col {}, n_uniq {}'.format(col, len(uniq), uniq))

--------------------------------------------------
# col fecha_dato, n_uniq 17


TypeError: '<' not supported between instances of 'NoneType' and 'str'

# テストデータ

In [ ]:
print(test.shape)
test.head(3)

In [ ]:
print(test.glimpse())